In [1]:
# Program to plot from the new CNAPS OPeNDAP server
#   using Hurricane Matthew as a guinea pig
#
# Joseph B Zambon
# 4 October 2016

#Dependencies
import pandas as pd
from pydap.client import open_url
import httplib2
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
# from mpl_toolkits.basemap import Basemap
import datetime
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import cmocean
from metpy.plots import ctables
import pyart

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Put your desired start/end dates in here.
#                              YYYY,MM,DD,HH
start_date = datetime.datetime(2019,8,27, 0)
end_date = datetime.datetime  (2019,9,7, 0)

# Put your desired color ranges in here.
slp_range = [960,1030]
sst_range = [14,32]
wnd_range = [0,30]
qv_spacing = 20
rain_range = [0,1]
mdbz_range = [0,80]
hwave_range = [0,10]


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# For inline plotting
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Link OPeNDAP datasets
wrf_cnaps_url = 'http://armstrong.meas.ncsu.edu:8080/thredds/dodsC/fmrc/useast_v2_WRF_full_L1/\
COAWST-WRF_Full_L1_FMRC_best.ncd'
wrf_dataset = open_url(wrf_cnaps_url)
print('Available WRF data:')
print(wrf_dataset.keys)
roms_cnaps_url = 'http://armstrong.meas.ncsu.edu:8080/thredds/dodsC/fmrc/useast_v2_ROMS_L1_s-coord/\
COAWST-ROMS_L1_Sigma-Coordinate_FMRC_best.ncd'
roms_dataset = open_url(roms_cnaps_url)
print('')
print('')
print('Available ROMS/SWAN data:')
print(roms_dataset.keys)

Available WRF data:
<bound method Mapping.keys of <DatasetType with children 'pressure', 'lat', 'lon', 'p_sfc', 'time', 'time_run', 'DateTime', 'year', 'month', 'day', 'hour', 'minute', 'Z_sfc', 'SST', 'T_sfc', 'slp', 'mdbz', 'T_2m', 'theta_2m', 'Td_2m', 'r_v_2m', 'q_2m', 'rh_2m', 'u_10m_tr', 'v_10m_tr', 'ws_10m', 'wd_10m', 'precip_g', 'precip_c', 'precip_fr', 'dryairmass', 'pblh', 'Z_p', 'T_p', 'theta_p', 'Td_p', 'rh_p', 'u_tr_p', 'v_tr_p', 'w_p', 'r_cloud_p', 'r_rain_p', 'r_ice_p', 'r_snow_p', 'pvo_p', 'avo_p', 'SW_d_acc', 'LW_d_acc', 'SW_u_acc', 'LW_u_acc', 'SW_d_toa_acc', 'LW_d_toa_acc', 'SW_u_toa_acc', 'LW_u_toa_acc', 'albedo', 'emiss_sfc', 'SH_acc', 'LH_acc', 'MH', 'u_star', 'time_offset'>>


Available ROMS/SWAN data:
<bound method Mapping.keys of <DatasetType with children 's_rho', 's_w', 'lon_rho', 'lat_rho', 'lon_u', 'lat_u', 'lon_v', 'lat_v', 'lon_psi', 'lat_psi', 'ocean_time', 'time', 'time_run', 'ntimes', 'ndtfast', 'dt', 'dtfast', 'dstart', 'nHIS', 'ndefHIS', 'nRST', 'Falp

In [4]:
# Let's ingest the latitude and longitude values
wrf_lon=np.array(wrf_dataset['lon'])
wrf_lat=np.array(wrf_dataset['lat'])
roms_lon=np.array(roms_dataset['lon_rho'])
roms_lat=np.array(roms_dataset['lat_rho'])
roms_mask=np.array(roms_dataset['mask_rho'])

In [5]:
#Find WRF time indices
wrf_origin_date = datetime.datetime(2019,5,21,0,0,0)

wrf_time=(np.array(wrf_dataset['time'][:])/24)+datetime.date.toordinal(wrf_origin_date)

wrf_start_index = np.where(wrf_time==datetime.date.toordinal(start_date))
wrf_start_index = wrf_start_index[0][0]
wrf_end_index = np.where(wrf_time==datetime.date.toordinal(end_date))
wrf_end_index=wrf_end_index[0][0]+1

# Print ordinal times to check against ROMS/SWAN output (should match)
# print(wrf_time[wrf_start_index:wrf_end_index])

In [6]:
#Find ROMS time indices
roms_origin_date = datetime.datetime(2019,5,21,0,0,0)
roms_time=(np.array(roms_dataset['time'][:]))/24+datetime.date.toordinal(roms_origin_date)

roms_start_index = np.where(roms_time==datetime.date.toordinal(start_date))
roms_start_index = roms_start_index[0][0]
roms_end_index = np.where(roms_time==datetime.date.toordinal(end_date))
roms_end_index=roms_end_index[0][0]+1

# Print ordinal times to check against WRF output (should match)
# print(roms_time[roms_start_index:roms_end_index])

extent = [float(np.min(wrf_lon)), float(np.max(wrf_lon)), float(np.min(wrf_lat)), float(np.max(wrf_lat))]


In [ ]:
#Make some plots
for t in range(0,roms_end_index-roms_start_index):
# for t in range(24,25):
    plt.figure(figsize=(14, 10),dpi=300)

    #Ingest data
    slp=np.array(wrf_dataset['slp'][int(wrf_start_index)+t,:,:])
    slp=np.squeeze(slp)
    sst=np.array(roms_dataset['temp'][int(roms_start_index)+t,35,:,:])
    sst=np.ma.array(sst,mask=np.isnan(sst))
    sst=np.squeeze(sst)
    u10=np.array(wrf_dataset['u_10m_tr'][int(wrf_start_index)+t,:,:])
    u10=np.squeeze(u10)
    v10=np.array(wrf_dataset['v_10m_tr'][int(wrf_start_index)+t,:,:])
    v10=np.squeeze(v10)
    wnd_mag = (u10**2 + v10**2 ) **0.5
    cp=np.array(wrf_dataset['precip_c'][int(wrf_start_index)+t,:,:])-\
       np.array(wrf_dataset['precip_c'][int(wrf_start_index)+t-1,:,:])
    cp=np.squeeze(cp)
    gp=np.array(wrf_dataset['precip_g'][int(wrf_start_index)+t,:,:])-\
       np.array(wrf_dataset['precip_g'][int(wrf_start_index)+t-1,:,:])
    gp=np.squeeze(gp)
    precip = (cp + gp) * 0.0393701  #convert to inches
    precip=np.ma.array(precip,mask=(precip<0.01))
    mdbz=np.array(wrf_dataset['mdbz'][int(wrf_start_index)+t,:,:])
    mdbz=np.squeeze(mdbz)
    mdbz=np.ma.array(mdbz,mask=(mdbz<10))
    wave=np.array(roms_dataset['Hwave'][int(roms_start_index)+t,:,:])
    wave=np.ma.array(wave,mask=np.isnan(wave))
    wave=np.squeeze(wave)

    date_t=datetime.date.fromordinal(wrf_time[wrf_start_index+t].astype(int))
    date_hrs=(wrf_time[wrf_start_index+t]-wrf_time[wrf_start_index+t].astype(int))*24
    date_hrs=date_hrs.astype(int)
    fore_valid = datetime.datetime(date_t.year,date_t.month,date_t.day,date_hrs)

    plt.suptitle('NCSU OOMG CNAPS Forecast: ' + fore_valid.strftime("%d %b %Y %H"+"UTC"),\
                 fontsize=28,family='Helvetica')

    ### Sea Level Pressure ###
    ax1 = plt.subplot(2, 3, 1, projection=ccrs.PlateCarree(central_longitude=np.mean(extent[0:2])))
    ax1.set_extent(extent,ccrs.PlateCarree())
    coastline = cfeature.NaturalEarthFeature(
            category='physical',  name='coastline',
            scale='50m', facecolor='none')
    ax1.add_feature(coastline, edgecolor='black', zorder=10)
    ax1.add_feature(cfeature.OCEAN,facecolor='#FFFFFF')
    ax1.add_feature(cfeature.LAKES,facecolor='None', edgecolor='black')
    ax1.add_feature(cfeature.LAND, edgecolor='black')
    ax1.add_feature(cfeature.RIVERS)
    ax1.add_feature(cfeature.BORDERS, zorder=1)
    states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces_lines',
            scale='50m', facecolor='none')
    ax1.add_feature(states_provinces, edgecolor='gray', zorder=10)
#     g1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                       linewidth=1, color='gray', alpha=0.2, linestyle='--')
#     g1.xlocator=mticker.FixedLocator(np.arange(-180,180.1,5))
#     g1.ylocator=mticker.FixedLocator(np.arange(-90,90.1,5))
#     g1.xlabels_top = False
#     g1.ylabels_left = False
    color = ax1.pcolormesh(wrf_lon,wrf_lat,slp[:,:],transform=ccrs.PlateCarree(),cmap='jet_r',\
                           vmin=slp_range[0],vmax=slp_range[1])
    plt.colorbar(color,orientation="horizontal", pad=0.05,ax=ax1)
    ax1.set_aspect('auto')
    plt.title('Sea Level Pressure (hPa)')
    plt.ylim(float(np.min(wrf_lat)), float(np.max(wrf_lat)))

    ### Sea Surface Temperature ###
    ax1 = plt.subplot(2, 3, 2, projection=ccrs.PlateCarree(central_longitude=np.mean(extent[0:2])))
    ax1.set_extent(extent)
    coastline = cfeature.NaturalEarthFeature(
            category='physical',  name='coastline',
            scale='50m', facecolor='none')
    ax1.add_feature(coastline, edgecolor='black', zorder=10)
    ax1.add_feature(cfeature.OCEAN,facecolor='#FFFFFF')
    ax1.add_feature(cfeature.LAKES,facecolor='None', edgecolor='black')
    ax1.add_feature(cfeature.LAND, edgecolor='black')
    ax1.add_feature(cfeature.RIVERS)
    ax1.add_feature(cfeature.BORDERS, zorder=1)
    states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces_lines',
            scale='50m', facecolor='none')
    ax1.add_feature(states_provinces, edgecolor='gray', zorder=10)
#     g1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                       linewidth=1, color='gray', alpha=0.2, linestyle='--')
#     g1.xlocator=mticker.FixedLocator(np.arange(-180,180.1,5))
#     g1.ylocator=mticker.FixedLocator(np.arange(-90,90.1,5))
#     g1.xlabels_top = False
#     g1.ylabels_left = False
    color = ax1.pcolormesh(roms_lon,roms_lat,sst[:,:],transform=ccrs.PlateCarree(),cmap=cmocean.cm.thermal,\
                           vmin=sst_range[0],vmax=sst_range[1])
    plt.colorbar(color,orientation="horizontal", pad=0.05,ax=ax1)
    ax1.set_aspect('auto')
    plt.title('Sea Surface Temperature ($^\circ$C)')
    plt.ylim(float(np.min(wrf_lat)), float(np.max(wrf_lat)))

    ### Wind Speed + Direction ###
    ax1 = plt.subplot(2, 3, 3, projection=ccrs.PlateCarree(central_longitude=np.mean(extent[0:2])))
    coastline = cfeature.NaturalEarthFeature(
            category='physical',  name='coastline',
            scale='50m', facecolor='none')
    ax1.add_feature(coastline, edgecolor='black', zorder=10)
    ax1.add_feature(cfeature.OCEAN,facecolor='#FFFFFF')
    ax1.add_feature(cfeature.LAKES,facecolor='None', edgecolor='black')
    ax1.add_feature(cfeature.LAND, edgecolor='black')
    ax1.add_feature(cfeature.RIVERS)
    ax1.add_feature(cfeature.BORDERS, zorder=1)
    states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces_lines',
            scale='50m', facecolor='none')
    ax1.add_feature(states_provinces, edgecolor='gray', zorder=10)
#     g1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                       linewidth=1, color='gray', alpha=0.2, linestyle='--')
#     g1.xlocator=mticker.FixedLocator(np.arange(-180,180.1,5))
#     g1.ylocator=mticker.FixedLocator(np.arange(-90,90.1,5))
#     g1.xlabels_top = False
#     g1.ylabels_left = False
    ax1.set_extent(extent)
    color = ax1.pcolormesh(wrf_lon,wrf_lat,wnd_mag[:,:],transform=ccrs.PlateCarree(),cmap=cmocean.cm.speed,\
                           vmin=wnd_range[0],vmax=wnd_range[1])
    plt.colorbar(color,orientation="horizontal", pad=0.05,ax=ax1)
    vector = ax1.quiver(wrf_lon[0:-1:qv_spacing,0:-1:qv_spacing],wrf_lat[0:-1:qv_spacing,0:-1:qv_spacing],\
                        u10[0:-1:qv_spacing,0:-1:qv_spacing],v10[0:-1:qv_spacing,0:-1:qv_spacing],\
                        transform=ccrs.PlateCarree())
    ax1.set_aspect('auto')
    plt.title('Wind Speed (m s$^-$$^1$)')
    plt.ylim(float(np.min(wrf_lat)), float(np.max(wrf_lat)))

    ### Precipitation ###
    ax1 = plt.subplot(2, 3, 4, projection=ccrs.PlateCarree(central_longitude=np.mean(extent[0:2])))
    ax1.set_extent(extent)
    coastline = cfeature.NaturalEarthFeature(
            category='physical',  name='coastline',
            scale='50m', facecolor='none')
    ax1.add_feature(coastline, edgecolor='black', zorder=10)
    ax1.add_feature(cfeature.OCEAN,facecolor='#FFFFFF')
    ax1.add_feature(cfeature.LAKES,facecolor='None', edgecolor='black')
    ax1.add_feature(cfeature.LAND, edgecolor='black')
    ax1.add_feature(cfeature.RIVERS)
    ax1.add_feature(cfeature.BORDERS, zorder=1)
    states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces_lines',
            scale='50m', facecolor='none')
    ax1.add_feature(states_provinces, edgecolor='gray', zorder=10)
#     g1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                       linewidth=1, color='gray', alpha=0.2, linestyle='--')
#     g1.xlocator=mticker.FixedLocator(np.arange(-180,180.1,5))
#     g1.ylocator=mticker.FixedLocator(np.arange(-90,90.1,5))
#     g1.xlabels_top = False
#     g1.ylabels_left = False
    color = ax1.pcolormesh(wrf_lon,wrf_lat,precip[:,:],transform=ccrs.PlateCarree(),cmap=cmocean.cm.rain,\
                           vmin=rain_range[0],vmax=rain_range[1])
    plt.colorbar(color,orientation="horizontal", pad=0.05,ax=ax1)
    ax1.set_aspect('auto')
    plt.title('Precipitation (inches)')
    plt.ylim(float(np.min(wrf_lat)), float(np.max(wrf_lat)))

    ### Simulated Radar Reflectivity ###
    ax1 = plt.subplot(2, 3, 5, projection=ccrs.PlateCarree(central_longitude=np.mean(extent[0:2])))
    ax1.set_extent(extent)
    coastline = cfeature.NaturalEarthFeature(
            category='physical',  name='coastline',
            scale='50m', facecolor='none')
    ax1.add_feature(coastline, edgecolor='black', zorder=10)
    ax1.add_feature(cfeature.OCEAN,facecolor='#FFFFFF')
    ax1.add_feature(cfeature.LAKES,facecolor='None', edgecolor='black')
    ax1.add_feature(cfeature.LAND, edgecolor='black')
    ax1.add_feature(cfeature.RIVERS)
    ax1.add_feature(cfeature.BORDERS, zorder=1)
    states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces_lines',
            scale='50m', facecolor='none')
    ax1.add_feature(states_provinces, edgecolor='gray', zorder=10)
#     g1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                       linewidth=1, color='gray', alpha=0.2, linestyle='--')
#     g1.xlocator=mticker.FixedLocator(np.arange(-180,180.1,5))
#     g1.ylocator=mticker.FixedLocator(np.arange(-90,90.1,5))
#     g1.xlabels_top = False
#     g1.ylabels_left = False
    cmap = 'pyart_NWSRef'
    levs = np.linspace(0,80,41,endpoint=True)
    ticks = np.linspace(0,80,9,endpoint=True)
    label = 'Radar Reflectivity Factor ($\mathsf{dBZ}$)'
    norm = mpl.colors.BoundaryNorm(levs,256)
    color = ax1.pcolormesh(wrf_lon,wrf_lat,mdbz[:,:],transform=ccrs.PlateCarree(),cmap=cmap,\
                           vmin=mdbz_range[0],vmax=mdbz_range[1])
    plt.colorbar(color,orientation="horizontal", pad=0.05,ax=ax1)
    ax1.set_aspect('auto')
    plt.title('Sim. Radar Reflectivity (dBZ)')
    plt.ylim(float(np.min(wrf_lat)), float(np.max(wrf_lat)))

    ### Significant Wave Height ###
    ax1 = plt.subplot(2, 3, 6, projection=ccrs.PlateCarree(central_longitude=np.mean(extent[0:2])))
    ax1.set_extent(extent)
    coastline = cfeature.NaturalEarthFeature(
            category='physical',  name='coastline',
            scale='50m', facecolor='none')
    ax1.add_feature(coastline, edgecolor='black', zorder=10)
    ax1.add_feature(cfeature.OCEAN,facecolor='#FFFFFF')
    ax1.add_feature(cfeature.LAND, edgecolor='black')
    ax1.add_feature(cfeature.LAKES,facecolor='None', edgecolor='black')
    ax1.add_feature(cfeature.RIVERS)
    ax1.add_feature(cfeature.BORDERS, zorder=1)
    states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces_lines',
            scale='50m', facecolor='none')
    ax1.add_feature(states_provinces, edgecolor='gray', zorder=10)
#     g1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                       linewidth=1, color='gray', alpha=0.2, linestyle='--')
#     g1.xlocator=mticker.FixedLocator(np.arange(-180,180.1,5))
#     g1.ylocator=mticker.FixedLocator(np.arange(-90,90.1,5))
#     g1.xlabels_top = False
#     g1.ylabels_left = False
    color = ax1.pcolormesh(roms_lon,roms_lat,wave[:,:],transform=ccrs.PlateCarree(),cmap=cmocean.cm.amp,\
                           vmin=hwave_range[0],vmax=hwave_range[1])
    plt.colorbar(color,orientation="horizontal", pad=0.05,ax=ax1)
    ax1.set_aspect('auto')
    plt.title('Significant Wave Height (m)')
    plt.ylim(float(np.min(wrf_lat)), float(np.max(wrf_lat)))

    plt.savefig('dorian_'+ str(t).zfill(2))

    

findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
